In [1]:
# Import des librairies
import pandas as pd 
import numpy as np 
from unidecode import unidecode 
from nltk.corpus import stopwords
import nltk
import re
import langid 
nltk.download('stopwords')
import simplemma
from collections import defaultdict

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## Insertion des données et vérification
df=pd.read_csv('df.csv')
df=pd.DataFrame(df,columns=['Date','Contenu','Retweet','Hashtags','utilisateur'])
df['Date']=pd.to_datetime(df['Date'])
print(df.shape)
print(df.dtypes)
print(df[:])

(482803, 5)
Date           datetime64[ns, UTC]
Contenu                     object
Retweet                      int64
Hashtags                    object
utilisateur                 object
dtype: object
                            Date  \
0      2022-10-15 23:56:38+00:00   
1      2022-10-15 23:52:53+00:00   
2      2022-10-15 23:49:40+00:00   
3      2022-10-15 23:47:44+00:00   
4      2022-10-15 23:47:17+00:00   
...                          ...   
482798 2022-02-01 01:38:34+00:00   
482799 2022-02-01 01:08:39+00:00   
482800 2022-02-01 00:37:04+00:00   
482801 2022-02-01 00:32:09+00:00   
482802 2022-02-01 00:20:38+00:00   

                                                  Contenu  Retweet  \
0       Cependant, le milliardaire aurait proposé un a...        0   
1       BREAKING ! Face au recul de l'armée russe en #...        0   
2       [Ukraine] Comment en est-on arrivé là ? Résumé...        0   
3       Mais le ministre rajoute " nous ne sommes pas ...        3   
4       @KalustD

In [3]:
#mise en place d'un classifieur de langue et mise en place sur notre dataframe
langue=[langid.classify(doc) for doc in df['Contenu']]
dflang=pd.DataFrame(langue, columns=['langue','score'])
df=pd.concat([df,dflang],axis=1)
df=df[df['langue']=='fr']
print(df['langue'].value_counts())
print(df.shape)

fr    471716
Name: langue, dtype: int64
(471716, 7)


In [4]:
### Traitement du corpus 
df['Contenu']=[str(doc) for doc in df['Contenu']]
df['Contenu']=[doc.lower() for doc in df['Contenu']]
df['Contenu']=[unidecode(doc) for doc in df['Contenu']]
df['Contenu'][:5]

/tmp/ipykernel_2441/2230237101.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df['Contenu'][:5]


0    cependant, le milliardaire aurait propose un a...
1    breaking ! face au recul de l'armee russe en #...
2    [ukraine] comment en est-on arrive la ? resume...
3    mais le ministre rajoute " nous ne sommes pas ...
4    @kalustd @guillaume_bigot @cnews @jchribuisson...
Name: Contenu, dtype: object

In [5]:
### Traitement des années et suppression des caracères spéciaux +suppression des URL.
df['Contenu']=[re.sub(r'[0-9]{4}','<annee>',doc) for doc in df['Contenu']]
df['Contenu']=[re.sub(r'[^a-z<>]+',' ',doc) for doc in df['Contenu']]
df['Contenu']=[re.sub(r'http*\S+', ' ', doc) for doc in df['Contenu']]
df['Contenu'][:5]

/tmp/ipykernel_2441/1950498021.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  df['Contenu'][:5]


0    cependant le milliardaire aurait propose un au...
1    breaking face au recul de l armee russe en ukr...
2     ukraine comment en est on arrive la resume de...
3    mais le ministre rajoute nous ne sommes pas en...
4     kalustd guillaume bigot cnews jchribuisson lo...
Name: Contenu, dtype: object

In [ ]:
### Insertion, traitement et suppression des stop_words en francais
stop_words=stopwords.words('french')
stop_words=[unidecode(sw) for sw in stop_words]
df['Contenu']=[' '.join([word for word in doc.split() if word not in stop_words]) for doc in df['Contenu']]
df['Contenu'][:5]

In [ ]:
### Visualisation de la fréquence d'apparition des mots

In [ ]:
corpus=df['Contenu']
words=' '.join(corpus)
words_list=words.split()

In [ ]:
freq=defaultdict(int)

for mot in words_list:
    freq[mot] +=1

In [ ]:
least=[ k for k in freq.keys()if freq[k]>6000] 
print (f" Voici la liste des mots avec une occurence > à 10000 : \n {least}")

In [ ]:
least=[ k for k in freq.keys()if freq[k]>12000] 
print (f" Voici la liste des mots avec une occurence > à 10000 : \n {least}")

La lémmatisation à une conservation du sens des mots plus élevé, dans notre projet on cherche à effectuer du topics modelling donc à
extraire des topics repprésentant au mieux les informations du corpus, c'est pourquoion choisis la lemmatisation 

In [ ]:
### Lémmatisation des données

In [ ]:
corpus=[' '.join([simplemma.lemmatize(word,lang='fr') for word in doc.split()]) for doc in corpus]
corpus[20:30]

In [ ]:
### Calcul de la taille du vocabulaire de corpus lemmatisé.
words=' '.join(corpus)
words_list=words.split()
vocab=list(set(words_list))
print(f"la taille du vocab est de : {len(vocab)}")

In [ ]:
### Calcul de la taille du vocabulaire non lemmatisé. 
words=' '.join(df['Contenu'])
words_list=words.split()
vocab=list(set(words_list))
print(f"la taille du vocab est de : {len(vocab)}")

In [ ]:
### Extraction des données lemmatisé.
df_lemm=pd.DataFrame(corpuslemm,columns=['Contenu'])
df_lemm.to_csv('df_lemm.csv')